In [2]:
# read data from csv file
import pandas as pd
import numpy as np
    
# read data from csv file
df = pd.read_csv('data.csv')

In [3]:
# For the search bar if the search bar doesn't work remove it
# Get the unique values of Titular
titular = df['Titular'].unique()
titular_list = list(titular)

In [4]:
# Take a 100 sample of df
df_sample = df.sample(100)

__Preprocessing__:

For a 100 sample takes 50 seconds

In [5]:
#pip install geopy
from geopy.geocoders import Nominatim

# Get the coordinates of a city (Municipio)

def get_coordinates(city):

    geolocator = Nominatim(user_agent="spain_explorer")
    location = geolocator.geocode(city) 
    latitude = location.latitude
    longitude = location.longitude
    return latitude, longitude

# Create a new dataframe with the coordinates

#df_sample['Latitude'], df_sample['Longitude'] = zip(*df_sample.apply(lambda x: get_coordinates(x['Provincia_FREQ']), axis=1))



In [6]:
l = df['Provincia_FREQ'].unique().tolist()
geo = []
for pro in l:
    geo.append(get_coordinates(pro))

In [19]:
a = pd.concat([pd.Series(l,name='provincias'),pd.Series(geo,name='coord')], axis=1)
a[['lat','long']] = pd.DataFrame(a['coord'].tolist(), index=a.index)
a = a.drop('coord', axis=1)
a.to_csv('provincias.csv', index=False)

In [21]:
# Municipio_Count is the number of radiolinks per Municipio

# Create a column 'Num_Radiolinks' 

df_sample['Num_Radiolinks'] = 1

# Group the rows of the dataframe by the 'Municipio' column and sum the 'Num_Radiolinks' column
df_agg = df_sample.groupby(['Provincia_FREQ'])['Num_Radiolinks'].sum().reset_index()

# Rename the column 'Num_Radiolinks' to the desired name
df_agg.rename(columns={'Num_Radiolinks': 'Municipio_Count'}, inplace=True)

# add the 'Municipio_Count' from df_agg to the df_sample joining by 'Municipio'

df_sample = pd.merge(df_sample, df_agg, on='Provincia_FREQ')

alejandro

In [30]:
import pandas as pd
import numpy as np
    
# read data from csv file
df = pd.read_csv('data.csv')
dfProvi = pd.read_csv('provincias.csv')
df = df.groupby(['Provincia_FREQ'], as_index=False)['Ref'].count().rename(columns={'Ref': 'Provincia_Count'})
df = pd.merge(df, dfProvi, left_on='Provincia_FREQ', right_on='provincias').drop('provincias', axis=1)
df.head()

,Provincia_FREQ,Provincia_Count,lat,long
0,ALBACETE,183,38.995092,-1.855915
1,ALICANTE/ALACANT,733,38.343637,-0.488171
2,ALMERÍA,417,36.841420,-2.462814
3,ARABA/ÁLAVA,290,42.844072,-2.682083
4,ASTURIAS,510,43.313387,-5.941920


__Map__


In [36]:
import folium
import pandas as pd

from folium.plugins import FastMarkerCluster
from folium import plugins

# Create a map centered on Spain
#spain_map = folium.Map(location=[40.416775, -3.703790], zoom_start=6)
spain_map = spain_map = folium.Map(location=[40.416775, -3.703790], zoom_start=6, tiles="Stamen Toner", control_scale=False)


# Create a linear color map for the number of Radiolinks per Municipio, with red for low values and green for high values
color_map = folium.LinearColormap(
    colors=['red', 'yellow', 'green'],
    vmin=df['Provincia_Count'].min(),
    vmax=df['Provincia_Count'].max()
)
# Add the color map to the map
color_map.add_to(spain_map)

# Create a layer for municipalities with the number of Radiolinks, and add it to the map
provincias = folium.FeatureGroup(name='Provincias')
for index, row in df.iterrows():
    folium.Circle(
        location=[row['lat'], row['long']],
        popup=f"Prvincia: {row['Provincia_FREQ']}<br>Number of Radiolinks: {row['Provincia_Count']}",
        radius=15000,
        fill=True,
        stroke=False,
        color=color_map(row['Provincia_Count']),
        fill_opacity=0.5
    ).add_to(provincias)

provincias.add_to(spain_map)

marker_cluster = FastMarkerCluster(
    data=df[['lat', 'long']].values.tolist(),
    popup=df.apply(lambda x: f"Provincia: {x['Provincia_FREQ']}<br>Number of Radiolinks: {x['Provincia_Count']}", axis=1).tolist()
).add_to(spain_map)

# show the map

spain_map